# Selective Areas

There are two useful sources of school based data routinely published by the DfE
1. "KS4_final" This is based on GCSE results of the last cohort to leave school.
   Due to COVID, the most recent is now from 2019.
2. "Schools Pupils & Characteristics" (SPC). This is based on census data so still up to date. Latest is available:
https://explore-education-statistics.service.gov.uk/find-statistics/school-pupils-and-their-characteristics
The exact location may change so manual instructions are to follow the links download the full data (40MB zip file)
extract this and set a path to the file
~\ancillary\spc_school_level_underlying_data.csv

Entire project is uploaded to https://github.com/jamescoombs3/grammars

In [1]:
import pandas as pd
import numpy as np

In [2]:
# set variable to wherever the DfE data has been saved
workdir = 'C:/_python/data/SPC/ancillary'
datafile = workdir + '/spc_school_level_underlying_data.csv'

In [3]:
# create a dictionary mapping DfE's headings of columns we want to keep against short names.
columns = {
    "URN": "URN",
    "LA name": "LA_name",
    "School Name": "SchName",
    "Sex of school description": "Sex",
    "Phase-type grouping": "PhaseGroup",
    "Admissions Policy": "AdmPolicy",
    "District Administrative name": "District",
    "Parl Con name": "Constituency",
    "Urban rural": "Urban",
    "School Postcode": "Postcode",
    "Academy_flag": "Academy_flag",
    "School size": "SchoolSize",
    "Number of key stage 3 pupils (years 7 to 9)": "KS3",
    "Number of key stage 4 pupils (years 10 and 11)": "KS4",
}

In [4]:
# Read the wanted columns into dataframe
spc = pd.read_csv(datafile, usecols=list(columns.keys()))
# Rename the columns
spc = spc.rename(columns=columns)

In [5]:
# Select just state funded secondary schools (removes primary schools, private schools, pupil referral units etc.)
spc = spc[spc['PhaseGroup'] == 'State-funded secondary']

# Add values in KS3 + KS4 to get total secondary school pupils on roll
spc['roll'] = spc['KS3'] + spc['KS4']

In [6]:
# The data contains four admissions policy types'; 'Selective', 'Non-selective', 'Not applicable' and 'Unknown'
# 'Selective' contains all 163 fully selective schools so the others are all non-selective.
spc = spc.replace({'Not applicable': 'Non-selective', 'Unknown': 'Non-selective'})

In [7]:
# Create two pivot tables to work out counts and proportions of grammar school places by
# 1. LA_Name
# 2. Constituency
LA_pivot = pd.pivot_table(spc, values='roll', index='LA_name', columns = ['AdmPolicy'], aggfunc=np.sum)
PC_pivot = pd.pivot_table(spc, values='roll', index='Constituency', columns = ['AdmPolicy'], aggfunc=np.sum)
LA_pivot.reset_index(inplace=True)
PC_pivot.reset_index(inplace=True)

In [8]:
# By default, the values are all floating point. Change them to integers 
LA_pivot['Non-selective'] = LA_pivot['Non-selective'].astype(int)
LA_pivot['Selective'] = LA_pivot['Selective'].fillna(0.0).astype(int)
PC_pivot['Non-selective'] = PC_pivot['Non-selective'].astype(int)
PC_pivot['Selective'] = PC_pivot['Selective'].fillna(0.0).astype(int)

In [9]:
# Add columns containing the percent proportion of selective places 
LA_pivot['PCT_Sel'] = LA_pivot['Selective'] / (LA_pivot['Selective'] + LA_pivot['Non-selective']) * 100
PC_pivot['PCT_Sel'] = PC_pivot['Selective'] / (PC_pivot['Selective'] + PC_pivot['Non-selective']) * 100

In [10]:
# sort the tables by most selective 
LA_pivot = LA_pivot.sort_values(by=['PCT_Sel'], ascending=False)
PC_pivot = PC_pivot.sort_values(by=['PCT_Sel'], ascending=False)

In [11]:
# Print top 10 Local Authorities with largest proportion of selective school places
LA_pivot.head(10).style.hide_index()

LA_name,Non-selective,Selective,PCT_Sel
Trafford,10187,6102,37.460863
Buckinghamshire,19536,11681,37.418714
Kent,60072,27801,31.637704
Southend-on-Sea,7830,3607,31.537991
Medway,11750,5262,30.931107
Torbay,5411,2322,30.027156
Wirral,13159,5272,28.603982
Sutton,11684,4497,27.791855
Slough,8586,3105,26.558891
Lincolnshire,30245,10478,25.729931


In [12]:
# Print top 10 Parliamentary Constituencies with largest proportion of selective school places
PC_pivot.head(10).style.hide_index()

Constituency,Non-selective,Selective,PCT_Sel
Maidstone and The Weald,4495,4503,50.044454
Wirral West,3194,2820,46.890589
Beaconsfield,2996,2473,45.218504
Tonbridge and Malling,4225,3260,43.553774
Wycombe,3689,2838,43.480925
Rochester and Strood,3655,2684,42.341063
Grantham and Stamford,4126,3005,42.139952
"Plymouth, Sutton and Devonport",2952,2122,41.821048
Altrincham and Sale West,5459,3591,39.679558
Southend West,4162,2707,39.408939
